In [1]:
from langchain import PromptTemplate        
from langchain.chains import RetrievalQA                
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter      #for creating chunks
from langchain.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from sentence_transformers import SentenceTransformer

from pinecone import Pinecone, ServerlessSpec
import os
from langchain_pinecone import PineconeVectorStore

/media/ashhar/New Volume/Internship/Medical-Chatbot-Using-Llama2/myenv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
# PINCONE_API_KEY="dfad884a-1a06-493c-850a-1de28db1b2dc"
# PINCONE_API_ENV=""
pc = Pinecone(api_key="dfad884a-1a06-493c-850a-1de28db1b2dc")

os.environ['PINECONE_API_KEY'] = 'dfad884a-1a06-493c-850a-1de28db1b2dc'

index_name= "medical-chatbot"

if index_name not in pc.list_indexes().names():
  pc.create_index(
  name="medical-chatbot",
  dimension=384,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

In [3]:
# Data extraction from pdf

def load_pdf(data):
    loader= DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents= loader.load()

    return documents

In [4]:
extracted_data = load_pdf("../data/")

In [5]:
# extracted_data

In [6]:
# Creating text Chunks so that the entire pdf is splitted into smaller texts

def text_split(extracted_data):
    # keeping size of each chunk to be 500 with overlap b/w chunks as 20
    text_splitter= RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap = 20)
    text_chunks= text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks=text_split(extracted_data)
print("No of chunks formed:", len(text_chunks))

No of chunks formed: 10484


In [8]:
# text_chunks

In [9]:
# Convert these chunks made into vector

# Download the embeddig model

def download_hugging_face_embedding():
    embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [10]:
embeddings= download_hugging_face_embedding()

/media/ashhar/New Volume/Internship/Medical-Chatbot-Using-Llama2/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [15]:
query_result = embeddings.embed_query("Hello World")
print("Length of embedding:", len(query_result))

Length of embedding: 384


In [16]:
sentences = ["Hello World"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings_test = model.encode(sentences)
print(embeddings_test)

/media/ashhar/New Volume/Internship/Medical-Chatbot-Using-Llama2/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[[-3.44772749e-02  3.10231671e-02  6.73496863e-03  2.61090044e-02
  -3.93619873e-02 -1.60302535e-01  6.69239834e-02 -6.44144043e-03
  -4.74505015e-02  1.47588691e-02  7.08753169e-02  5.55275641e-02
   1.91933028e-02 -2.62513645e-02 -1.01094991e-02 -2.69404948e-02
   2.23073885e-02 -2.22266335e-02 -1.49692625e-01 -1.74930710e-02
   7.67623540e-03  5.43523207e-02  3.25447111e-03  3.17259654e-02
  -8.46213996e-02 -2.94060130e-02  5.15956469e-02  4.81240712e-02
  -3.31480778e-03 -5.82792200e-02  4.19692881e-02  2.22107433e-02
   1.28188848e-01 -2.23389454e-02 -1.16562592e-02  6.29283637e-02
  -3.28763090e-02 -9.12260413e-02 -3.11753135e-02  5.26995584e-02
   4.70348373e-02 -8.42030272e-02 -3.00561991e-02 -2.07448304e-02
   9.51779820e-03 -3.72177688e-03  7.34331086e-03  3.93243134e-02
   9.32740495e-02 -3.78857949e-03 -5.27421013e-02 -5.80582209e-02
  -6.86439266e-03  5.28326817e-03  8.28930065e-02  1.93627477e-02
   6.28449675e-03 -1.03308028e-02  9.03239194e-03 -3.76837887e-02
  -4.52060

In [13]:
query_result   # Vector representation of "Hello World"

[-0.03447727486491203,
 0.031023167073726654,
 0.00673496862873435,
 0.026109004393219948,
 -0.03936198726296425,
 -0.16030253469944,
 0.06692398339509964,
 -0.006441440433263779,
 -0.04745050147175789,
 0.01475886907428503,
 0.07087531685829163,
 0.05552756413817406,
 0.01919330283999443,
 -0.02625136449933052,
 -0.010109499096870422,
 -0.026940494775772095,
 0.022307388484477997,
 -0.02222663350403309,
 -0.1496926248073578,
 -0.017493071034550667,
 0.0076762354001402855,
 0.05435232073068619,
 0.0032544711139053106,
 0.031725965440273285,
 -0.08462139964103699,
 -0.02940601296722889,
 0.051595646888017654,
 0.04812407121062279,
 -0.003314807778224349,
 -0.05827922001481056,
 0.041969288140535355,
 0.022210743278265,
 0.1281888484954834,
 -0.022338945418596268,
 -0.011656259186565876,
 0.06292836368083954,
 -0.032876309007406235,
 -0.09122604131698608,
 -0.031175313517451286,
 0.052699558436870575,
 0.04703483730554581,
 -0.08420302718877792,
 -0.030056199058890343,
 -0.02074483036994

In [47]:

# index = pc.list_indexes().names()

# # Function to check if vectors exist in the index
# def vectors_exist(index_name, vector_ids):
#     try:
#         # Fetch the existing vectors by their IDs
#         existing_vectors = index_name.fetch(ids=vector_ids)
#         return len(existing_vectors["vectors"]) > 0
#     except Exception as e:
#         print(f"Error checking vectors: {e}")
#         return False

# if index_name not in pc.list_indexes().names():
#     vectorstore_from_docs = PineconeVectorStore.from_documents(
#         text_chunks,
#         index_name=index_name,
#         embedding=embeddings
#     )

In [48]:
# made_index=pc.describe_index("medical-chatbot")
# len(made_index)

In [49]:
# converting and uploading vectors to pincone index

# vectorstore_from_docs = PineconeVectorStore.from_documents(
#         text_chunks,
#         index_name=index_name,
#         embedding=embeddings
#     )

In [17]:
# Successfully stored in Pincone Vector DB
# vectorstore_from_docs
index_name="medical-chatbot"

vectorstore_from_docs = PineconeVectorStore.from_existing_index(index_name, embeddings)

In [18]:
# Searching for a query and giving top 3 results (using cosine similarity) from the stored vectors [Ranked Results]

query= "What are Allergies?"
answer=vectorstore_from_docs.similarity_search(query, k=3)
print("Result obtained:",answer)

Result obtained: [Document(page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 2591Physical allergy', metadata={'page': 297.0, 'source': 'data/Medical-Book.pdf'}), Document(page_content='KEY TERMS\nAllergen —Any substance that irritates only those\nwho are sensitive (allergic) to it.\nAsthma —Wheezing (labored breathing) due to\nallergies or irritation of the lungs.\nDecongestant —Medicines that shrink blood ves-\nsels and consequently mucus membranes. Pseu-doephedrine, phenylephrine, and phenylpropano-lamine are the most common.\nSinus —Air-filled cavities surrounding the eyes and', metadata={'page': 22.0, 'source': 'data/Medical-Book.pdf'}), Document(page_content='allergic or bacterial rhinitis and sinusitis. An over-the-counter antihistamine such as Actifed may provide relief.\nIf allergies cause anosmia, adjustments should be\nmade to avoid allergens. If dust causes allergies, careshould be taken to clean areas such as the bedroom.\nAntibiotics may be prescribed for infections.\nOther me

In [19]:
# Now to have a correct answer from top 3 answers to a query, we use our LLM model

# Creating a prompt
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [20]:

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [21]:
try:
    llm = CTransformers(
        model='../model/llama-2-7b-chat.ggmlv3.q2_K.bin',
        model_type="llama",
    )
    print("Model loaded successfully.")
except Exception as e:
    print(f"Failed to load the model: {e}")



Model loaded successfully.


In [22]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever= vectorstore_from_docs.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [23]:
while True:
    user_input = input("Input Prompt: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting the loop.")
        break
    result = qa({"query": user_input})
    print("Response: ", result["result"])

/media/ashhar/New Volume/Internship/Medical-Chatbot-Using-Llama2/myenv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response:  Whata DiaRhoria?
DiaRhoria is a genus of protozoa that includes Entamoeba
histolytica, Gparydia lamblia, and other species.
Exiting the loop.
